Terrawise use the following image data over areas of interest: Dynamic World yearly maps, NDVI, NDSI, RGB.
This notebook generates and displays NDVI, NDSI and RGB based on sentinel-2 data pulled from AWS. It also reads continental-scale SLGA COGs and clips to the aoi for display.


In [79]:
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio as rs
from rasterio.features import geometry_window, geometry_mask
from rasterio.plot import reshape_as_raster
from rasterio.windows import transform

from dash import Dash, html, dcc, callback, Output, Input, dash_table
import dash_bootstrap_components as dbc
import leafmap as leafmap
import dash_leaflet
import plotly.express as px
import pystac_client

In [80]:
# Sentinel-2 AWS archive
output_dir = 'data/temp'
geom_path = 'data/stac-demo-aoi.geojson'
sentinel_search_url = 'https://earth-search.aws.element84.com/v1'
timerange = '2023-10-01/2023-10-30'
dst_crs = "EPSG:3857"


geom = gpd.read_file(geom_path)
bbox = geom.geometry.total_bounds.tolist()

print("Geojson geometry:\n ",geom)
print("Bounding box: \n ",bbox)


Geojson geometry:
                                              geometry
0  POLYGON ((145.36955 -38.17343, 145.36955 -38.1...
Bounding box: 
  [145.36954762848688, -38.18996780887601, 145.3950650720538, -38.1734250469396]


In [81]:
# Perform STAC filtering to get Sentinel-2 data
try:
    catalog = pystac_client.Client.open(sentinel_search_url)
    results = catalog.search(
        max_items=3,
        bbox=bbox,
        datetime=timerange,
        query={"eo:cloud_cover": {"lt": 50}},
        collections=["sentinel-2-l2a"]
    ).item_collection()
except pystac_client.exceptions.APIError as e:
    print(f"Error querying STAC API: {e}")

print(results[0])

<Item id=S2B_55HCT_20231029_0_L2A>


In [82]:
for item in results:
    # Process each STAC item and calculate NDVI
    raster_crs = 'epsg:' + str(item.properties['proj:epsg'])
    polygon_gdf = geom.to_crs(str(raster_crs))
    #read in bands needed for indices:
    """
    Red and nir bands for calculating NDVI
    green and swir2 bands for NDSI
    visual band is a pre-made visual image
    """
    rgb_href = item.assets['visual'].href
    red_href = item.assets['red'].href
    green_href = item.assets['green'].href
    nir_href = item.assets['nir'].href
    swir2_href = item.assets['swir2'].href

    with rs.open(nir_href) as nir_src, rs.open(red_href) as red_src:
        for feature in polygon_gdf.iterfeatures(show_bbox=True):
            window = geometry_window(nir_src, [feature["geometry"]])
            window_transform = transform(window, nir_src.transform)
            window_shape = (window.height, window.width)
            

            # Read all the data in the window, masking out any NoData
            nir = nir_src.read(window=window, masked=True).astype('float32')
            red = red_src.read(window=window, masked=True).astype('float32')

            # Update the NoData mask to exclude anything outside the polygon
            mask = geometry_mask([feature["geometry"]], window_shape, window_transform)
            nir.mask += mask
            red.mask += mask

            # Calculate NDVI
            ndvi = (nir - red) / (nir + red)

            # Save the masked NDVI to one tif per polygon
            ndvi_path = '/tmp/stac-ndvi.tif'

            meta = nir_src.meta

            meta.update({"driver": "GTiff", 
                            "dtype": ndvi.dtype, 
                            "height": window.height, 
                            "width":window.width, 
                            "transform": window_transform,
                            "count":1})
            
            with rs.open(output_dir + '/test-ndvi.tif', 'w', **meta) as dst:
                dst.write(ndvi)

            ndvi_s3key = 'stac-ndvi.tif'